In [1]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
from glob import glob as glob
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
gsname = 'gs1109'

### load data

In [13]:
## run make_df-1102.py to make dfs
datadf = pd.read_csv('data/%s-datadf.csv'%gsname)


In [16]:
gsdf = pd.read_csv('data/%s-summdf.csv'%gsname).astype(float)
gsdf

,Unnamed: 0,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,skipt1,testacc-b,testacc-e,testacc-i,testacc-l,testacc-m
0,0.0,0.073630,6.370319,79.826294,0.832120,0.0,1.0,1.0,1.0,1.0,0.498417,0.499789,0.501165,0.502152,0.498899
1,0.0,0.348616,13.871358,26.964805,0.004257,0.0,1.0,1.0,1.0,0.0,0.614382,0.602276,0.728084,0.726727,0.728537
2,0.0,1.024693,91.419792,68.490744,1.071253,0.0,1.0,1.0,1.0,1.0,0.499932,0.502340,0.499738,0.499662,0.500587
3,0.0,1.255439,25.583928,54.100463,0.271345,0.0,1.0,1.0,1.0,1.0,0.501458,0.499396,0.499260,0.499357,0.500229
4,0.0,1.368341,0.262132,96.339771,1.121625,0.0,1.0,1.0,1.0,0.0,0.499360,0.501192,0.499417,0.501721,0.499536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510549,0.0,99.414639,87.020214,45.508947,0.575661,0.0,1.0,1.0,1.0,0.0,0.500000,0.500000,0.500000,0.500000,0.500000
510550,0.0,99.464559,5.426427,43.342461,0.714906,0.0,1.0,1.0,1.0,0.0,0.500000,0.500000,0.500000,0.500000,0.500000
510551,0.0,99.679125,27.433675,14.528440,0.273414,0.0,1.0,1.0,1.0,0.0,0.500000,0.500000,0.500000,0.500000,0.500000
510552,0.0,99.771709,97.062026,70.541135,0.767256,0.0,1.0,1.0,1.0,0.0,0.500000,0.500000,0.500000,0.500000,0.500000


In [17]:
# datadf.columns
# paramL = ['concentration', 'stickiness_wi', 'stickiness_bt',
#        'sparsity', 'pvar', 'lrate', 'lratep', 'decay_rate']

### compute df metrics, form summary df

In [20]:
# subset of dataframe containing relevant params
Btest = gsdf.loc[:,'testacc-b'] > 0.95
Itest1 = gsdf.loc[:,'testacc-i'] > 0.5
Itest2 = gsdf.loc[:,'testacc-i'] < 0.55
Etest = gsdf.loc[:,'testacc-e'] > 0.8
Mtest1 = gsdf.loc[:,'testacc-m'] > 0.8
Mtest2 = gsdf.loc[:,'testacc-m'] < 0.9
Ltest = gsdf.loc[:,'testacc-l'] > 0.6
MItest = gsdf.loc[:,'testacc-m'] - gsdf.loc[:,'testacc-i'] > 0.1
LItest = gsdf.loc[:,'testacc-l'] - gsdf.loc[:,'testacc-i'] > 0.1

C = Btest&Itest1&Itest2&Mtest2
sgsdf = gsdf[C]
sgsdf.sort_values('testacc-l',ascending=False)

,Unnamed: 0,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,skipt1,testacc-b,testacc-e,testacc-i,testacc-l,testacc-m
390649,0.0,3.497113,20.131674,64.120315,0.089724,0.0,1.0,1.0,1.0,1.0,0.952022,0.952022,0.539269,0.614728,0.574630
242895,0.0,2.519468,80.139590,80.716372,0.024391,0.0,1.0,1.0,1.0,1.0,0.952425,0.952424,0.544383,0.597294,0.574442
122799,0.0,5.307060,58.853608,94.753300,0.051546,0.0,1.0,1.0,1.0,1.0,0.952259,0.952258,0.538654,0.594312,0.568405
72971,0.0,2.766957,50.783868,77.302032,0.036299,0.0,1.0,1.0,1.0,1.0,0.952352,0.952352,0.547404,0.581468,0.581873
24295,0.0,1.463721,48.715566,71.697747,0.019428,0.0,1.0,1.0,1.0,1.0,0.952455,0.952455,0.549586,0.580800,0.576972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93060,0.0,5.518016,6.080470,39.456943,0.317581,0.0,1.0,1.0,1.0,1.0,0.950625,0.950612,0.501471,0.483759,0.497995
267036,0.0,7.581236,10.692653,73.556375,0.409647,0.0,1.0,1.0,1.0,1.0,0.950041,0.950043,0.502469,0.481926,0.508206
114580,0.0,8.599427,6.971903,76.379451,0.403167,0.0,1.0,1.0,1.0,1.0,0.950073,0.950066,0.500413,0.480114,0.500060
422804,0.0,7.746292,9.182396,97.428663,0.392933,0.0,1.0,1.0,1.0,1.0,0.950145,0.950136,0.509884,0.474046,0.499945


In [25]:
## select single param for plotting full acc trace
row = sgsdf.sort_values('testacc-l',ascending=False).iloc[0]
row.astype(float)

Unnamed: 0        0.000000
concentration     3.497113
stickiness_wi    20.131674
stickiness_bt    64.120315
sparsity          0.089724
pvar              0.000000
lrate             1.000000
lratep            1.000000
decay_rate        1.000000
skipt1            1.000000
testacc-b         0.952022
testacc-e         0.952022
testacc-i         0.539269
testacc-l         0.614728
testacc-m         0.574630
Name: 390649, dtype: float64

### plt single condition

In [ ]:
## form query string for above row
query_str = " and ".join(
  ["%s == %f"%(k,v) for k,v in dict(row.astype(float)[1:10].round(4)).items()]
)
print(query_str)
## df of single parameter condition`
cond_df = datadf.round(4).query(query_str)

concentration == 3.497100 and stickiness_wi == 20.131700 and stickiness_bt == 64.120300 and sparsity == 0.089700 and pvar == 0.000000 and lrate == 1.000000 and lratep == 1.000000 and decay_rate == 1.000000 and skipt1 == 1.000000


In [ ]:
paramstr = "__".join(["%s-%s"%(k,v) for k,v in cond_df.iloc[0][1:12].items()])
paramstr

In [ ]:
f,axar = plt.subplots(1,1,figsize=(20,8))
ax=axar
for c,cdf in cond_df.groupby('cond'):
  ax.plot(np.arange(200),cdf.acc,label=c)
ax.set_ylim(0,1)
plt.legend()
plt.axhline(0.5,c='k',ls='--')
plt.title("%s\n%s"%(paramstr[:94],paramstr[94:]))
plt.savefig('figures/analysis_gs/acc-%s.png'%paramstr)